In [1]:
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from scipy.misc import imread, imsave, imresize
import pandas as pd
import numpy as np
import subprocess
import glob
import re
import matplotlib.pyplot as plt

from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Conv2D, AtrousConvolution2D, Flatten, Dense, MaxPooling2D, Dropout, ZeroPadding2D, Activation
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.callbacks import EarlyStopping

Using Theano backend.


ImportError: DLL load failed: 找不到指定的程序。

In [3]:
data_dir = "data/set2/"
data_type = {"Filename": str, "Genres": str, "Release Year": int}
albums = pd.read_csv(data_dir + "albumlabel.csv", dtype=data_type, parse_dates=["Release Year"])
# albums['Artist'] = [name.split('_')[1] for name in albums['Filename']]
# albums['NewFileName'] = albums['Artist'] +'_'+ albums['Release Year'].map(lambda x: str(x)[:10])
albums.head()

,Filename,Genres,Release Year
0,12 X 5_1964,rock pop bluesrock,1964-01-01 00:00:00
1,The Rolling Stones_1964,rock blues pop poprock rhythmblues rockroll,1964-01-01 00:00:00
2,It's All Over Now_1964,rock rhythmblues classicrock,1964-01-01 00:00:00
3,December's Children (And Everybody's)_1966,rock bluesrock garagerock poprock,1966-01-01 00:00:00
4,No. 2_1965,rock bluesrock rockroll,1965-01-01 00:00:00


In [ ]:
# Label to index number New genres (12 groups)
token = Tokenizer()
genres = ['rock', 'metal', 'punk', 'pop', 'blue', 'funk', 'jazz', 'electro', 'hiphop', 'rap', 'country', 'other']
token.fit_on_texts(genres)
for k in token.word_index:
    token.word_index[k] -= 1

idx_word = dict()
for key in token.word_index.keys():
    idx_word[token.word_index[key]] = key

label_lst = albums.Genres.get_values()
data_y = np.zeros((len(label_lst), max(token.word_index.values())+1))
for i, album_labels in enumerate(label_lst):
    splt_labels = album_labels.split()
    for label in splt_labels:
        nl = [l for l in token.word_index if l in label]
        if len(nl) > 0:
            for j in nl:
                data_y[i, token.word_index[j]] = 1
        else:
            data_y[i, token.word_index['other']] = 1
print data_y[:5,]

In [ ]:
# Read image
X_origin = []
dropName = []
for filename in albums['Filename'].get_values():
    try:
        X_origin.append(np.array(Image.open(data_dir+'resize/'+filename+'.jpg')))
    except:
        dropName.append(filename)
X_origin = np.array(X_origin)

X = np.zeros((X_origin.shape[0], 32, 32, 3))
for i in xrange(X_origin.shape[0]):
    X[i] = X_origin[i]

dropIndex = [i for i, name in enumerate(albums['Filename']) if name in dropName]
data_y = np.delete(data_y, dropIndex, axis=0)

In [ ]:
optimizer = 'adadelta'
objective = 'binary_crossentropy'

def center_normalize(x):
    return (x-K.mean(x))/K.std(x)

def single2multi(x):
    return x/np.max(x)

# split data
train_x, test_x, train_y, test_y = train_test_split(X, data_y, test_size=0.2)

In [ ]:
# Build NN Model (model to optimize)
model = Sequential()
model.add(Activation(activation=center_normalize,input_shape=train_x.shape[1:]))

model.add(ZeroPadding2D((1,1),input_shape=train_x.shape[1:], dim_ordering='tf'))
model.add(Conv2D(64, 3, 3, border_mode='valid', activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(64, 3, 3, border_mode='valid', activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2,2)))

model.add(ZeroPadding2D((1,1),input_shape=train_x.shape[1:], dim_ordering='tf'))
model.add(Conv2D(128, 3, 3, border_mode='valid', activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, 3, 3, border_mode='valid', activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2,2)))

model.add(ZeroPadding2D((1,1),input_shape=train_x.shape[1:], dim_ordering='tf'))
model.add(Conv2D(256, 3, 3, border_mode='valid', activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, 3, 3, border_mode='valid', activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2,2)))

model.add(ZeroPadding2D((1,1),input_shape=train_x.shape[1:], dim_ordering='tf'))
model.add(Conv2D(256, 3, 3, border_mode='valid', activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, 3, 3, border_mode='valid', activation='relu'))
model.add(MaxPooling2D((2, 2), strides=(2,2)))

model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(train_y.shape[1], activation='sigmoid'))
# model.add(Dense(train_y.shape[1], activation=single2multi))

model.compile(loss=objective,
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1, mode='auto')        
model.fit(train_x, train_y,
          validation_data = (test_x, test_y),
          batch_size=20,
          nb_epoch=10,
          class_weight='auto',
          verbose=1,
          callbacks=[early_stopping])

In [ ]:
def greater50percent(y):
    p = []
    for s in y:
        tmp = []
        for i in xrange(len(s)):
            if s[i] > 0.3:
                tmp.append(i)
        p.append(tmp)
    return p

pred = model.predict(test_x)
p  = greater50percent(pred)
p_ = greater50percent(test_y)

In [ ]:
for i in xrange(len(p)):
    print(i, p[i], p_[i])